In [27]:
from PIL import Image, ImageDraw, ImageFilter
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import keras
from keras import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, Convolution2D,AveragePooling2D,MaxPooling2D,AveragePooling2D,Flatten,Dropout
from keras import optimizers
from tqdm import tqdm
import matplotlib.patches as patches
plt.rcParams["figure.figsize"] = (8,6)
from keras.utils import plot_model
from keras.utils.np_utils import to_categorical  

In [28]:
(_, _), (x_neg, _) = keras.datasets.cifar100.load_data()
x_neg=[x_neg[2*i] for i in range(int(len(x_neg)/2))]

### Red

In [29]:
folder='../data/red'
ims=list(filter(lambda x: x[0] != '.', os.listdir(folder)))

x=[np.load(folder+'/image_'+str(i)+'.npy') for i in range(int(len(ims)/2))]
y=[np.load(folder+'/values_'+str(i)+'.npy')for i in range(int(len(ims)/2))]

x_r=x#.astype('float32')/255
y_r = y#.astype('float32')/max(canvas_size)

### White

In [30]:
folder='../data/white'
ims=list(filter(lambda x: x[0] != '.', os.listdir(folder)))

x=[np.load(folder+'/image_'+str(i)+'.npy') for i in range(int(len(ims)/2))]
y=[np.load(folder+'/values_'+str(i)+'.npy')for i in range(int(len(ims)/2))]

x_w=x#.astype('float32')/255
y_w = y#.astype('float32')/max(canvas_size)

### Yellow

In [31]:
folder='../data/yellow'
ims=list(filter(lambda x: x[0] != '.', os.listdir(folder)))

x=[np.load(folder+'/image_'+str(i)+'.npy') for i in range(int(len(ims)/2))]
y=[np.load(folder+'/values_'+str(i)+'.npy')for i in range(int(len(ims)/2))]

x_y=x#.astype('float32')/255
y_y = y#.astype('float32')/max(canvas_size)

### Blue

In [32]:
folder='../data/blue'
ims=list(filter(lambda x: x[0] != '.', os.listdir(folder)))

x=[np.load(folder+'/image_'+str(i)+'.npy') for i in range(int(len(ims)/2))]
y=[np.load(folder+'/values_'+str(i)+'.npy')for i in range(int(len(ims)/2))]
x_b=x#.astype('float32')/255
y_b = y#.astype('float32')/max(canvas_size)

In [33]:
x_pos=[]
x_tf=x_b+x_r+x_w+x_y
y_tf=y_b+y_r+y_w+y_y
for i in tqdm(range(len(x_tf))):
    y=y_tf[i]
    x=Image.fromarray(x_tf[i][int(y[2]):int(y[3]),int(y[0]):int(y[1])]).resize((32,32))
    x_pos+=[np.array(x)]

100%|██████████| 4799/4799 [00:02<00:00, 2026.32it/s]


In [37]:
x_train,y_train,x_test,y_test=[],[],[],[]

In [38]:
for i in tqdm(range(len(x_pos))):
    if random.random()<0.9:
        x_train+=[np.array(x_pos[i])/255]
        y_train+=[1]
    else:
        x_test+=[np.array(x_pos[i])/255]
        y_test+=[1]

100%|██████████| 4799/4799 [00:00<00:00, 28395.14it/s]


In [39]:
for i in tqdm(range(len(x_neg))):
    if random.random()<0.9:
        x_train+=[np.array(x_neg[i])/255]
        y_train+=[0]
    else:
        x_test+=[np.array(x_neg[i])/255]
        y_test+=[0]

100%|██████████| 5000/5000 [00:00<00:00, 40550.53it/s]


In [42]:
model = Sequential()
model.add(Convolution2D(30, (3,3),activation='relu',input_shape=(32,32,3)))
model.add(Convolution2D(30, (3,3),activation='relu'))
model.add(Convolution2D(30, (3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.build()

In [43]:
np.array(y_train).shape

(8833,)

In [44]:
model.compile(optimizer='adam',loss='mse',metrics=["acc"])

In [ ]:
model.fit(np.array(x_train), np.array(y_train), epochs=10,
                  validation_split=0.1, shuffle=True,
                  verbose=1)

Train on 7949 samples, validate on 884 samples
Epoch 1/10
7949/7949 [==============================] - 24s 3ms/step - loss: 0.1224 - acc: 0.8278 - val_loss: 0.0532 - val_acc: 0.9378
Epoch 2/10
7949/7949 [==============================] - 24s 3ms/step - loss: 0.0556 - acc: 0.9257 - val_loss: 0.0461 - val_acc: 0.9480
Epoch 3/10
7949/7949 [==============================] - 24s 3ms/step - loss: 0.0386 - acc: 0.9509 - val_loss: 0.0169 - val_acc: 0.9796
Epoch 4/10
7949/7949 [==============================] - 25s 3ms/step - loss: 0.0303 - acc: 0.9615 - val_loss: 0.0151 - val_acc: 0.9808
Epoch 5/10
7949/7949 [==============================] - 22s 3ms/step - loss: 0.0246 - acc: 0.9679 - val_loss: 0.0385 - val_acc: 0.9457
Epoch 6/10
7949/7949 [==============================] - 22s 3ms/step - loss: 0.0189 - acc: 0.9763 - val_loss: 0.0298 - val_acc: 0.9638
Epoch 7/10
7949/7949 [==============================] - 23s 3ms/step - loss: 0.0163 - acc: 0.9804 - val_loss: 0.0530 - val_acc: 0.9310
Epoch 8/

In [121]:
model.evaluate(np.array(x_test), np.array(y_test))

968/968 [==============================] - 1s 655us/step


[0.022121560221968355, 0.9700413346290588]

In [122]:
model.save('../trained_models/binaryclassifier.h5')